In [1]:
import rosbag
PATH="/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/tro2020-bags/pre/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
input_bag = PATH +"drone1-swarm_local_2020-12-13-21-31-51.bag"
output_bag = PATHOUT + "drone1-swarm_local_2020-12-13-21-31-51_processed.bag"
moveout_topics = {
    "/swarm_drones/node_detected",
    "/swarm_drones/swarm_drone_fused",
    "/swarm_drones/swarm_drone_fused_relative",
    "/swarm_drones/swarm_frame",
    "/swarm_drones/swarm_frame_predict"
}
#Moveout node_detected
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        if topic in moveout_topics:
            pass
        else:
            #outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)
            outbag.write(topic, msg, t)            

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


FileNotFoundError: [Errno 2] No such file or directory: '/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/tro2020-bags/pre/drone1-swarm_local_2020-12-13-21-31-51.bag'

In [4]:
import rosbag
import os
import rospy
#PATH="/home/xuhao/bags/swarm_local_2020_12/"
#PATH="/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/2021-3-1-outdoor/trial3/"
PATHOUT="/home/xuhao/bags/swarm_local_2021-3-2/trial3/"
bags = ["drone1.bag",
                "drone2.bag",
                "drone2.bag"
       ]
t_offset = 10
#t_offset = 90

t0 = 0

for bag in bags:
    for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
        print(f"Bag {bag} start at {t.to_sec()}")
        if t.to_sec() > t0:
            t0 = t.to_sec()
            break
        else:
            break
print("Will use", t0, "as start")

ros_t0 = None
lps_t1 = None
moveout_topics = {
    "/uwb_node/incoming_broadcast_data"
}

dts = {bags[0]:0}

for topic, msg, t in rosbag.Bag(PATH+bags[0]).read_messages():
    if t.to_sec() > t0 + t_offset and topic == "/uwb_node/time_ref":
        _t0 = msg.header.stamp
        _t1 = msg.time_ref
        dt0 = _t0 - _t1
        break

for bag_name in bags:
    for topic, msg, t in rosbag.Bag(PATH+bag_name).read_messages():
        if  t.to_sec() > t0 + t_offset and topic == "/uwb_node/time_ref":
            _t0 = msg.header.stamp
            _t1 = msg.time_ref
            dt1 = _t0 - _t1
            break
    dt = (dt1 - dt0)
    print(f"Dt between drones {dt.to_sec()} bag {bag_name}", dt1, dt0)
    dts[bag_name] = dt

for bag in bags:
    print(f"Converting {bag}")
    #cmd = f"/opt/ros/melodic/bin/rosbag filter {PATH}{bag} {PATHOUT}filtered-{bag} 't.to_sec() >= {t}'"
    #print(os.popen(cmd).read())
    _dt = rospy.Time(0)
    if bag != bags[0]:
        _dt = dts[bag]
        #dt = 0
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if topic in moveout_topics:
                pass
            else:
                t = msg.header.stamp - _dt if msg._has_header else t - _dt
                if msg._has_header:
                    msg.header.stamp = msg.header.stamp - _dt
                if t.to_sec() > t0 + t_offset:
                    outbag.write(topic, msg, t)

Bag drone1.bag start at 1614678030.9677083
Bag drone2.bag start at 1614678029.0100212
Will use 1614678030.9677083 as start
Dt between drones 0.0 bag drone1.bag 1614674811598963544 1614674811598963544
Dt between drones 1.262796552 bag drone2.bag 1614674812861760096 1614674811598963544
Converting drone1.bag
Converting drone2.bag


In [ ]:
ros_t0 = None
lps_t1 = None
for bag in bags:
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if t < t0:
                continue
            if bag ==  "drone2-swarm_local_raw_2020-12-12.bag":
                if topic == "/uwb_node/remote_nodes":
                    if ros_t0 is None:
                        ros_t0 = msg.header.stamp.to_sec()
                    if lps_t1 is None:
                        lps_t1 = msg.sys_time
                    lps_time = (msg.header.stamp.to_sec() - ros_t0)*1000 + lps_t1
                    #print(msg.sys_time, lps_time)
                    msg.sys_time = int(lps_time)
            outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)

In [3]:
for bag in bags:
    print(f"Converting {bag}")
    #cmd = f"/opt/ros/melodic/bin/rosbag filter {PATH}{bag} {PATHOUT}filtered-{bag} 't.to_sec() >= {t}'"
    #print(os.popen(cmd).read())
    _dt = rospy.Time(0)
    if bag != bags[0]:
        _dt = dts[bag]
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if topic in moveout_topics:
                pass
            else:
                t = msg.header.stamp - _dt if msg._has_header else t - _dt
                if msg._has_header:
                    msg.header.stamp = msg.header.stamp - _dt
                if t.to_sec() > t0 + t_offset:
                    outbag.write(topic, msg, t)

Converting drone1.bag
Converting drone2.bag
